In [1]:
from classes.ConditionalGAN import CWGAN
from utils.callbacks import WandbImagesConditionalGAN, SaveGeneratorWeights
import tensorflow as tf
from tensorflow import keras
import numpy as np
import wandb
from wandb.keras import WandbCallback
import os
import tensorflow as tf

wandb.login()


wandb: Currently logged in as: matteoferrante (use `wandb login --relogin` to force relogin)


True

In [18]:

config={"dataset":"mnist","type":"CW-GAN"}

wandb.init(project="TorVergataExperiment-Generative",config=config)

## DATA


(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


###
BS=256
#strategy = tf.distribute.MirroredStrategy()

#with strategy.scope():
g=CWGAN(latent_dim=100,target_shape=(28,28,1),d_steps=5)

wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [19]:
g.discriminator.summary()

Model: "discriminator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 1, 50)        500         ['input_15[0][0]']               
                                                                                                  
 dense_8 (Dense)                (None, 1, 784)       39984       ['embedding_4[0][0]']            
                                                                                                  
 input_16 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                      

In [20]:
g.gan.build(input_shape=(None,100))
g.gan.summary()

Model: "gan"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 generator (Functional)         (None, 28, 28, 1)    587256      ['input_21[0][0]',               
                                                                  'input_20[0][0]']               
                                                                                                  
 discriminator (Functional)     (None, 1)            1229093     ['generator[0][0]',            

In [21]:

ts=len(x_train)//BS

x_train=np.expand_dims(x_train.astype("float32")/255.,-1)
x_test=np.expand_dims(x_test.astype("float32")/255.,-1)

y_train=np.expand_dims(y_train.astype("int32"),-1)
y_test=np.expand_dims(y_test.astype("int32"),-1)


train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_dataset=  train_dataset.shuffle(buffer_size=1024).batch(BS).repeat()


test_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_dataset=test_dataset.shuffle(1024).batch(BS)


In [22]:

os.makedirs("models/cwgan_mnist",exist_ok=True)
model_check=SaveGeneratorWeights(filepath="models/cwgan_mnist/generator_cwgan_mnist.h5")



callbacks=[
    WandbImagesConditionalGAN(target_shape=(28,28,1)),
    WandbCallback(),
    model_check
]



In [ ]:
g.train(train_dataset,steps_per_epoch=ts,epochs=40,log=False,callbacks=callbacks)


[EPOCH] 0/40


 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 44/46 [00:38<00:01,  1.14it/s]